In [ ]:
# %% Deep learning - Section 13.132
#    APRF example 1: mnist

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [1]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F
import pandas              as pd
import scipy.stats         as stats
import sklearn.metrics     as skm
import time

from torch.utils.data                 import DataLoader,TensorDataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')


In [2]:
# %% Get the data

# Load data
data_all = np.loadtxt(open('sample_data/mnist_train_small.csv','rb'),delimiter=',')

# Remove labels (i.e., numbers IDs) from dataset
labels = data_all[:,0]
data   = data_all[:,1:]

# Normalize to [0,1]
data_norm = data / np.max(data)

# Covert to tensor
data_T   = torch.tensor(data_norm).float()
labels_T = torch.tensor(labels).long()

# Split data with scikitlearn
train_data,test_data, train_labels,test_labels = train_test_split(data_T,labels_T,test_size=0.1)

# PyTorch datasets
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# DataLoader objects
batch_size   = 32
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])


In [44]:
# %% Model class

def gen_model():

    class model(nn.Module):
        def __init__(self):
            super().__init__()

            # Architecture
            self.input  = nn.Linear(784,64 )
            self.hid1   = nn.Linear( 64,32 )
            self.hid2   = nn.Linear( 32,32 )
            self.output = nn.Linear( 32,10 )

        def forward(self,x):
            x = F.relu(self.input(x))
            x = F.relu(self.hid1(x))
            x = F.relu(self.hid2(x))

            return self.output(x)

    # Model instance, loss function, and optimizer
    ANN       = model()
    loss_fun  = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(ANN.parameters(),lr=0.01)

    return ANN,loss_fun,optimizer


In [45]:
# %% Function to train the model

def train_model():

    # Epochs (few to keep some varaibility in performace) and fresh model instance
    num_epochs = 10
    ANN,loss_fun,optimizer = gen_model()

    # Preallocate vars
    losses    = torch.zeros(num_epochs)
    train_acc = torch.zeros(num_epochs)
    test_acc  = torch.zeros(num_epochs)

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Loop over training data batches
        batch_acc  = []
        batch_loss = []

        for X,y in train_loader:

            # Forward pass, backpropagation, and optimizer step
            yHat = ANN(X)
            loss = loss_fun(yHat,y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Loss and accuracy from this batch
            batch_loss.append(loss.item())
            batch_acc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )

        losses[epoch_i]    = np.mean(batch_loss).item()
        train_acc[epoch_i] = np.mean(batch_acc).item()

        # Test accuracy
        ANN.eval()

        with torch.no_grad():
            X,y = next(iter(test_loader))
            yHat = ANN(X)
            test_acc[epoch_i] = 100*torch.mean((torch.argmax(yHat,axis=1)==y).float())

        ANN.train()

    return train_acc,test_acc,losses,ANN


In [46]:
# %% Fit the model

train_acc,test_acc,losses,ANN = train_model()


In [ ]:
# %% Plotting

phi = (1 + np.sqrt(5)) / 2
fig,ax = plt.subplots(1,2,figsize=(1.5*phi*6,6))

ax[0].plot(losses)
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')
ax[0].set_ylim([0,3])
ax[0].set_title('Model loss')

ax[1].plot(train_acc,label='Train')
ax[1].plot(test_acc,label='Test')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy (%)')
ax[1].set_ylim([10,100])
ax[1].set_title(f'Final model test accuracy: {test_acc[-1]:.2f}%')
ax[1].legend()

plt.savefig('figure6_aprf_example_mnist.png')

plt.show()

files.download('figure6_aprf_example_mnist.png')


In [ ]:
# %% Get data to compute performance measures on train and test data

# Predictions for (all) training data (i.e. raw output of last layer)
yHat        = ANN(train_loader.dataset.tensors[0])
train_preds = torch.argmax(yHat,axis=1)
print(train_preds.shape)

# Predictions for test data (i.e. raw output of last layer)
yHat       = ANN(test_loader.dataset.tensors[0])
test_preds = torch.argmax(yHat,axis=1)
print(test_preds.shape)


In [ ]:
# %% Interlude on computing performance metrics in multiclass data

# For the MNIST dataset, we have 10 classes, so we need 10 performance values,
# be it accuracy, precision, recall, F1, or others; let's consider here
# precision as example

# Option 1: compute precision for each class (i.e., each digit)
precision_1 = skm.precision_score(train_loader.dataset.tensors[1],train_preds,average=None)
print(precision_1)

# Option 2: compute average precision, weighted by N
precision_2 = skm.precision_score(train_loader.dataset.tensors[1],train_preds,average='weighted')
print(precision_2)

# Option 3: compute average precision, unweighted (same as 'weighted' if category samples are equal)
precision_3 = skm.precision_score(train_loader.dataset.tensors[1],train_preds,average='macro')
print(precision_3)


In [50]:
# %% Compute performance measures on train and test data

# Preallocate
train_metrics = np.zeros(4)
test_metrics  = np.zeros(4)

# Training performance measures (accuracy is already an overall measure)
train_metrics[0] = skm.accuracy_score (train_loader.dataset.tensors[1],(train_preds).float())
train_metrics[1] = skm.precision_score(train_loader.dataset.tensors[1],(train_preds).float(),average='weighted')
train_metrics[2] = skm.recall_score   (train_loader.dataset.tensors[1],(train_preds).float(),average='weighted')
train_metrics[3] = skm.f1_score       (train_loader.dataset.tensors[1],(train_preds).float(),average='weighted')

# Test performance measures
test_metrics[0] = skm.accuracy_score (test_loader.dataset.tensors[1],(test_preds).float())
test_metrics[1] = skm.precision_score(test_loader.dataset.tensors[1],(test_preds).float(),average='weighted')
test_metrics[2] = skm.recall_score   (test_loader.dataset.tensors[1],(test_preds).float(),average='weighted')
test_metrics[3] = skm.f1_score       (test_loader.dataset.tensors[1],(test_preds).float(),average='weighted')


In [ ]:
# %% Plotting

phi = (1 + np.sqrt(5)) / 2
fig = plt.figure(figsize=(6*phi,6))

plt.bar(np.arange(4)-.1,train_metrics,.5)
plt.bar(np.arange(4)+.1,test_metrics,.5)
plt.xticks([0,1,2,3],['Accuracy','Precision','Recall','F1-score'])
plt.ylim([.5,.7])
plt.legend(['Train','Test'])
plt.title('Performance metrics')

plt.savefig('figure7_aprf_example_mnist.png')

plt.show()

files.download('figure7_aprf_example_mnist.png')


In [ ]:
# %% Check for possible biases towards certain digits

# Class-specific precision and recall for test data
precision = skm.precision_score(test_loader.dataset.tensors[1],test_preds,average=None)
recall    = skm.recall_score   (test_loader.dataset.tensors[1],test_preds,average=None)

# Plotting
phi = (1 + np.sqrt(5)) / 2
fig = plt.figure(figsize=(1.5*6*phi,6))

plt.bar(np.arange(10)-.15,precision,.5)
plt.bar(np.arange(10)+.15,recall,.5)
plt.xticks(range(10),range(10))
plt.ylim([.1,1])
plt.xlabel('Digit')
plt.legend(['Precision','Recall'])
plt.title('Category-specific performance metrics')

plt.savefig('figure8_aprf_example_mnist.png')

plt.show()

files.download('figure8_aprf_example_mnist.png')


In [ ]:
# %% Potting

# Confusion matrices
train_conf = skm.confusion_matrix(train_loader.dataset.tensors[1],train_preds,normalize='true')
test_conf  = skm.confusion_matrix(test_loader.dataset.tensors[1],test_preds,normalize='true')

phi = (1 + np.sqrt(5)) / 2
fig,ax = plt.subplots(1,2,figsize=(phi*6,6),constrained_layout=True)

# Training confusion matrix
ax[0].imshow(train_conf,'Blues',vmax=.1)
ax[0].set_xticks(range(10))
ax[0].set_yticks(range(10))
ax[0].set_title('TRAIN confusion matrix')
ax[0].set_xlabel('True number')
ax[0].set_xlabel('Predicted number')
ax[0].set_ylabel('True number')

# Test confusion matrix
img = ax[1].imshow(test_conf,cmap='Blues',vmax=.1)
ax[1].set_xticks(range(10))
ax[1].set_yticks(range(10))
ax[1].set_title('TEST confusion matrix')
ax[1].set_xlabel('Predicted number')
ax[1].set_ylabel('True number')
fig.colorbar(img,ax=ax[1],shrink=0.6)

plt.savefig('figure9_aprf_example_mnist.png')

plt.show()

files.download('figure9_aprf_example_mnist.png')


In [ ]:
# %% Exercise 1
#    The Adam optimizer is pretty amazing, isn't it? Change the code to get the overall test accuracy between 50% and 80%.
#    You can consider changing the optimizer, learning rate, and number of epochs. Then show the performance metrics.
#    Are there systematic difficulties with some numbers, or simply a general decline in performance overall?

# Trying with SGD, lr=0.001 and 25 epochs. If we look at the overall pattern of
# APRF, it seems like the performance is going down in general; when looking at
# precision and recall for each digit, however, the picture changes a lot, the
# metrics oscillates between ~0.1 and ~0.9, with some digits doing particularly
# bad in general, some doing worse on the precision (e.g., 0, 6, 7), and some
# doing worse on the recall (e.g., 2, 3, 4)


In [ ]:
# %% Exercise 2
#    Are the y-axis and color-limits still appropriate? Modify the visualization code so that the graphs are adaptive
#    to the numerical values of the performance metrics and confusion matrices.

# That can be done by flexibly setting plt.ylim([]) and vmin/vmax. Note how on
# the confusion matrices one can now see better the deflection in performance
# for some of the digits (e.g., 2, 3, 4)

# Plot 1
phi = (1 + np.sqrt(5)) / 2
fig = plt.figure(figsize=(6*phi,6))

plt.bar(np.arange(4)-.1,train_metrics,.5)
plt.bar(np.arange(4)+.1,test_metrics,.5)
plt.xticks([0,1,2,3],['Accuracy','Precision','Recall','F1-score'])
ymin = min(train_metrics.min(), test_metrics.min())
ymax = max(train_metrics.max(), test_metrics.max())
plt.ylim([ymin*0.95,ymax*1.05])
plt.legend(['Train','Test'])
plt.title('Performance metrics')

plt.savefig('figure14_aprf_example_mnist_extra2.png')

plt.show()

files.download('figure14_aprf_example_mnist_extra2.png')

# Plot 2
precision = skm.precision_score(test_loader.dataset.tensors[1],test_preds,average=None)
recall    = skm.recall_score   (test_loader.dataset.tensors[1],test_preds,average=None)

phi = (1 + np.sqrt(5)) / 2
fig = plt.figure(figsize=(1.5*6*phi,6))

plt.bar(np.arange(10)-.15,precision,.5)
plt.bar(np.arange(10)+.15,recall,.5)
plt.xticks(range(10),range(10))
ymin = min(precision.min(),recall.min())
ymax = max(precision.max(),recall.max())
plt.ylim([ymin*0.95,ymax*1.05])
plt.xlabel('Digit')
plt.legend(['Precision','Recall'])
plt.title('Category-specific performance metrics')

plt.savefig('figure15_aprf_example_mnist_extra2.png')

plt.show()

files.download('figure15_aprf_example_mnist_extra2.png')

# Plot 3
train_conf = skm.confusion_matrix(train_loader.dataset.tensors[1],train_preds,normalize='true')
test_conf  = skm.confusion_matrix(test_loader.dataset.tensors[1],test_preds,normalize='true')

phi = (1 + np.sqrt(5)) / 2
fig,ax = plt.subplots(1,2,figsize=(phi*6,6),constrained_layout=True)

vmin = min(train_conf.min(),test_conf.min())
vmax = max(train_conf.max(),test_conf.max())

ax[0].imshow(train_conf,'Blues',vmin=vmin,vmax=vmax)
ax[0].set_xticks(range(10))
ax[0].set_yticks(range(10))
ax[0].set_title('TRAIN confusion matrix')
ax[0].set_xlabel('True number')
ax[0].set_xlabel('Predicted number')
ax[0].set_ylabel('True number')

img = ax[1].imshow(test_conf,cmap='Blues',vmin=vmin,vmax=vmax)
ax[1].set_xticks(range(10))
ax[1].set_yticks(range(10))
ax[1].set_title('TEST confusion matrix')
ax[1].set_xlabel('Predicted number')
ax[1].set_ylabel('True number')
fig.colorbar(img,ax=ax[1],shrink=0.6)

plt.savefig('figure16_aprf_example_mnist_extra2.png')

plt.show()

files.download('figure16_aprf_example_mnist_extra2.png')
